In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from gensim.models import Word2Vec
import re

In [24]:
data = pd.read_csv('data.csv')

print(data.head())

   id                                           consulta          intencion
0   1   ¿Cuál es la fecha límite para el proyecto Final?     fechas_entrega
1   2   ¿Cuáles son los temas a tratar en la clase de...            temario
2   3   ¿Dónde puedo encontrar material de estudio ad...   material_estudio
3   4   ¿Cuál es el horario de consulta del profesor ...           horarios
4   5                     ¿Cómo implemento un árbol AVL?    ejemplos_codigo


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         102 non-null    int64 
 1   consulta   102 non-null    object
 2   intencion  102 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [26]:
data.columns

Index(['id', 'consulta', 'intencion'], dtype='object')

In [27]:
# removiendo el espacio de las columnas
data.columns = data.columns.str.strip()
data['consulta'] = data['consulta'].str.strip()
data['intencion'] = data['intencion'].str.strip()

print(data.values)

[[1 '¿Cuál es la fecha límite para el proyecto Final?' 'fechas_entrega']
 [2 '¿Cuáles son los temas a tratar en la clase de esta semana?'
  'temario']
 [3 '¿Dónde puedo encontrar material de estudio adicional?'
  'material_estudio']
 [4
  '¿Cuál es el horario de consulta del profesor de Programación Avanzada?'
  'horarios']
 [5 '¿Cómo implemento un árbol AVL?' 'ejemplos_codigo']
 [6 '¿Hay alguna práctica programada para el próximo fin de semana?'
  'horarios']
 [7
  '¿Dónde puedo encontrar ejemplos de código para algoritmos de ordenamiento eficientes?'
  'ejemplos_codigo']
 [8 '¿Cuál es el cronograma de exámenes para este semestre?' 'cronograma']
 [10 '¿Cuándo es la entrega de la implementación de árboles rojinegros?'
  'fechas_entrega']
 [11
  '¿Dónde puedo encontrar información sobre la solución del ejercicio de backtracking?'
  'material_estudio']
 [12
  '¿Hay algún repositorio de código disponible para la práctica de pilas y colas en Python?'
  'repositorio']
 [13
  '¿Cómo puedo ha

In [28]:
#sort data by intencion
data = data.sort_values(by=['intencion'], ascending=True)
print(data.head())

    id                                           consulta   intencion
50  52  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
43  45  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma
28  30  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
27  29  ¿Cuál es el cronograma de entregas para las ta...  cronograma
52  54  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma


In [29]:
# Eliminando los signos de puntuacion
data['consulta'] = data['consulta'].str.replace(r'[^\w\s]', '', regex=True)

In [30]:
train_txt,test_txt,train_label,test_labels = train_test_split(data['consulta'],data['intencion'],test_size = 0.2)

In [31]:
# Convertir el texto en una lista de palabras
words = []
for text in data['consulta']:
    words += text_to_word_sequence(text)

# Contar el número de palabras únicas
unique_words = set(words)
num_unique_words = len(unique_words)

# Imprimir el número de palabras únicas
print("Número de palabras únicas: ", num_unique_words)

Número de palabras únicas:  211


In [32]:
max_num_words = 350 #el numero de palabras unicas es de 211 entonces agregamos un maximo con holgura
classes = np.unique(data['intencion'])

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_txt)
word_index = tokenizer.word_index

word_index

{'de': 1,
 'el': 2,
 'la': 3,
 'puedo': 4,
 'dónde': 5,
 'para': 6,
 'encontrar': 7,
 'es': 8,
 'cuál': 9,
 'del': 10,
 'horario': 11,
 'en': 12,
 'las': 13,
 'pa': 14,
 'cronograma': 15,
 'repositorio': 16,
 'fecha': 17,
 'límite': 18,
 'los': 19,
 'avanzada': 20,
 'ejemplos': 21,
 'temario': 22,
 'entrega': 23,
 'este': 24,
 'programación': 25,
 'quiero': 26,
 'que': 27,
 'al': 28,
 'actualizado': 29,
 'con': 30,
 'código': 31,
 'proyecto': 32,
 'árboles': 33,
 'se': 34,
 'práctica': 35,
 'cuatrimestre': 36,
 'temas': 37,
 'específicos': 38,
 'semana': 39,
 'sesiones': 40,
 'tutoría': 41,
 'tp': 42,
 'cómo': 43,
 'final': 44,
 'próximo': 45,
 'necesito': 46,
 'enlace': 47,
 'programacion': 48,
 'exámenes': 49,
 'obtener': 50,
 'una': 51,
 'copia': 52,
 'cuáles': 53,
 'son': 54,
 'repaso': 55,
 'información': 56,
 'sobre': 57,
 'clases': 58,
 'consulta': 59,
 'mi': 60,
 'curso': 61,
 'hay': 62,
 'examen': 63,
 'trabajo': 64,
 'por': 65,
 'mes': 66,
 'python': 67,
 'horarios': 68,
 'im

In [33]:
ls=[]
for c in train_txt:
    ls.append(len(c.split()))   #cada consulta se convierte en una lista de palabras y se cuenta el numero de palabras

maxLen=int(np.percentile(ls, 98))   #se calcula el percentil 98 de la lista de palabras

train_sequences = tokenizer.texts_to_sequences(train_txt)   #convierte las consultas en secuencias de tokens
print("train sequence tokenice:\n",train_sequences)

train_sequences = pad_sequences(train_sequences, maxlen=maxLen, padding='post')     #rellena con ceros las secuencias para que todas tengan la misma longitud dada por el percentil 98
print("train sequence pad sequence:\n",train_sequences)

test_sequences = tokenizer.texts_to_sequences(test_txt)
print("test sequence tokenice:\n",test_sequences)
test_sequences = pad_sequences(test_sequences, maxlen=maxLen, padding='post')
print("test sequence tokenice:\n",test_sequences)

train sequence tokenice:
 [[5, 4, 7, 2, 16, 29, 30, 19, 21, 1, 31, 98, 99], [5, 4, 7, 100, 101, 6, 2, 32, 1, 33, 12, 67], [26, 19, 68, 102, 1, 24, 36], [53, 54, 19, 37, 38, 27, 34, 103, 12, 2, 22, 1, 3, 104, 39], [46, 2, 22], [5, 4, 7, 2, 11, 1, 13, 40, 1, 41, 6, 2, 55, 1, 25, 20], [5, 4, 7, 56, 57, 13, 105, 1, 23], [9, 8, 3, 17, 18, 6, 3, 23, 10, 32], [5, 4, 7, 2, 11, 1, 13, 58, 1, 106], [5, 4, 7, 2, 11, 41, 6, 42], [107, 8, 3, 23, 1, 3, 69, 1, 33, 108], [70, 21, 1, 31, 6, 33, 109], [46, 71, 28, 16, 1, 14], [70, 2, 47, 28, 16, 1, 14], [9, 8, 2, 11, 1, 13, 40, 1, 35], [5, 4, 7, 2, 11, 1, 3, 59, 6, 2, 55, 1, 72, 1, 14], [26, 19, 68, 1, 41, 1, 14], [46, 2, 110, 28, 16, 1, 14], [43, 4, 111, 60, 73, 1, 112, 12, 113, 6, 114, 115], [5, 4, 7, 2, 11, 29, 1, 13, 40, 1, 41], [9, 8, 2, 11, 1, 116, 1, 3, 41, 6, 2, 32, 44], [9, 8, 2, 47, 28, 22, 74, 10, 61, 1, 48, 20, 1, 24, 75], [26, 2, 22, 29], [117, 2, 15, 1, 49, 6, 2, 45, 118, 76, 12, 2, 61, 1, 14], [43, 4, 119, 120, 3, 121, 1, 122, 12, 60, 123

In [34]:
label_encoder = LabelEncoder() # codifica las clases en numeros
integer_encoded = label_encoder.fit_transform(classes) # codifica las clases en numeros

print("Clases: ", classes)
print("Clases codificadas: ", integer_encoded)

Clases:  ['cronograma' 'ejemplos_codigo' 'fechas_entrega' 'horarios'
 'material_estudio' 'repositorio' 'temario']
Clases codificadas:  [0 1 2 3 4 5 6]


In [35]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

print(integer_encoded)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]]


In [36]:
onehot_encoder.fit(integer_encoded)

print("Onehot catgories:",onehot_encoder.categories_) # codifica las clases en numeros

Onehot catgories: [array([0, 1, 2, 3, 4, 5, 6])]


c:\Users\Marcos\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [37]:
train_label

29         repositorio
20    material_estudio
83            horarios
68             temario
85             temario
            ...       
12    material_estudio
60          cronograma
74             temario
84             temario
72      fechas_entrega
Name: intencion, Length: 81, dtype: object

In [38]:
train_label_encoded = label_encoder.transform(train_label) # codifica las clases en numeros

train_label_encoded

array([5, 4, 3, 6, 6, 3, 2, 2, 3, 3, 2, 1, 5, 5, 3, 3, 3, 3, 4, 3, 3, 6,
       6, 0, 4, 2, 2, 0, 3, 4, 2, 0, 0, 2, 2, 3, 0, 5, 0, 5, 3, 5, 6, 6,
       3, 2, 5, 4, 6, 1, 0, 0, 4, 3, 1, 0, 2, 3, 5, 0, 1, 3, 5, 3, 4, 5,
       3, 2, 6, 4, 6, 2, 3, 5, 0, 1, 4, 0, 6, 6, 2])

In [39]:
train_label_encoded = train_label_encoded.reshape(len(train_label_encoded), 1)

train_label_encoded

array([[5],
       [4],
       [3],
       [6],
       [6],
       [3],
       [2],
       [2],
       [3],
       [3],
       [2],
       [1],
       [5],
       [5],
       [3],
       [3],
       [3],
       [3],
       [4],
       [3],
       [3],
       [6],
       [6],
       [0],
       [4],
       [2],
       [2],
       [0],
       [3],
       [4],
       [2],
       [0],
       [0],
       [2],
       [2],
       [3],
       [0],
       [5],
       [0],
       [5],
       [3],
       [5],
       [6],
       [6],
       [3],
       [2],
       [5],
       [4],
       [6],
       [1],
       [0],
       [0],
       [4],
       [3],
       [1],
       [0],
       [2],
       [3],
       [5],
       [0],
       [1],
       [3],
       [5],
       [3],
       [4],
       [5],
       [3],
       [2],
       [6],
       [4],
       [6],
       [2],
       [3],
       [5],
       [0],
       [1],
       [4],
       [0],
       [6],
       [6],
       [2]])

In [40]:
train_label = onehot_encoder.transform(train_label_encoded)    # codifica las clases en numeros

train_label

array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
 

In [41]:
#ahora lo mismo pero para los datos de testeo

test_labels_encoded = label_encoder.transform(test_labels)
test_labels_encoded = test_labels_encoded.reshape(len(test_labels_encoded), 1)

print("pre-onehot:\n",test_labels)

test_labels = onehot_encoder.transform(test_labels_encoded)

print("\n\npos-onehot:\n",test_labels)

pre-onehot:
 1              temario
86          cronograma
81      fechas_entrega
59             temario
6      ejemplos_codigo
97            horarios
64            horarios
90            horarios
25             temario
38          cronograma
26             temario
78         repositorio
92          cronograma
53         repositorio
36             temario
94     ejemplos_codigo
95      fechas_entrega
32      fechas_entrega
2     material_estudio
69          cronograma
13      fechas_entrega
Name: intencion, dtype: object


pos-onehot:
 [[0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [

In [43]:
words

['dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'prácticas',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'evaluaciones',
 'para',
 'el',
 'próximo',
 'año',
 'académico',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'entregas',
 'para',
 'las',
 'tareas',
 'restantes',
 'del',
 'semestre',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'practicas',
 'de',
 'pa',
 'programación',
 'avanzada',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'actualizado',
 'de',
 'evaluaciones',
 'de',
 'programacion',
 'avanzada',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'pa',
 'de',
 'este',
 'cuatrimestre',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'de

In [46]:
w2v_model = Word2Vec(sentences=[words], vector_size=100, window=5, min_count=1, workers=4)
"""
    Parametros:
    sentences: lista de lista de palabras
    size: dimension del vector (posible parametro a modificar 100-300)
    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)
    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)
    workers: numero de hilos de ejecucion
"""


'\n    Parametros:\n    sentences: lista de lista de palabras\n    size: dimension del vector (posible parametro a modificar 100-300)\n    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)\n    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)\n    workers: numero de hilos de ejecucion\n'

In [53]:
# mostramos los embeddings de la palabra "hola"
print(w2v_model.wv['dónde'])

[-9.2231408e-03  4.7834730e-03 -8.2930280e-03 -1.6092287e-03
  5.0338195e-03 -1.0380825e-03  4.1080848e-03  1.0306911e-02
 -8.2265800e-03  4.3195765e-03 -7.4328827e-03 -7.7517319e-04
 -8.4470846e-03  4.5720413e-03 -3.8525849e-03 -7.4047204e-03
 -1.4502687e-03  3.1906622e-03  4.7205286e-03 -1.3973443e-02
  1.4518070e-03 -6.2836739e-03  1.0256744e-02  8.5055986e-03
 -3.2795351e-03  1.2684301e-03 -1.1400800e-03  4.3563684e-03
 -1.1708008e-02  6.6899665e-04  1.0694679e-02  2.4324651e-03
  4.3680011e-03 -1.3705671e-02  5.6707393e-03 -1.5508276e-04
  1.3191091e-04  1.8351773e-03 -1.5997282e-03 -3.9339345e-03
  3.1392740e-03 -1.1795669e-02 -1.1673977e-02  7.2277579e-03
  9.7305411e-03 -6.1917901e-03  6.9202455e-05 -7.9927000e-04
  7.1327738e-03 -5.8215763e-03  5.8255824e-03  2.3192461e-03
  1.0962043e-02 -4.7695665e-03 -1.5980708e-03 -6.6363574e-03
 -7.1133762e-03 -9.8364791e-03 -4.3229843e-03 -4.8678573e-03
  5.6163482e-03 -6.9436939e-03  4.2905966e-03 -1.2481523e-04
 -5.3811269e-03  3.26550

In [54]:
# mostramos el tamaño del vocabulario
print("Tamaño del vocabulario: ", len(w2v_model.wv))

Tamaño del vocabulario:  211
